In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


import xml.etree.ElementTree as ET
import os
import csv
import json
import requests
import pyodbc
import urllib3
import time


SInit = time.time()

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Получение директории программы
basedir = os.path.abspath(os.getcwd())

# Подключение к БД
connectionstring = r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + basedir + r'\BFO_DB.accdb;'
connection_sql = pyodbc.connect(connectionstring)
cursor = connection_sql.cursor()

#файлы подключения с листом inn и выгрузкой баговых компаний
Name_Org_Actual = 'OrgList.txt'
Name_Org_Bad = 'BadOrg.txt'
Name_Org_Bad_inn = 'BadOrgInn.txt'
Data_org = 'DataOrg.txt'


def get_external_data_by_inn(inn): # Основная процедура выборки данных с сайта БФО ФНС
    
    Error = 0   #обработка ошибок выкачки
    
    cookies_get_org_id = {
        '_ym_uid': '1670252833970654852',
        '_ym_d': '1670252833',
        '_ym_isad': '2',
        'disclaimed': 'true',
    }
    headers_get_org_id = {
        'Accept': '*/*',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-GB;q=0.8,en;q=0.7,en-US;q=0.6',
        'Connection': 'keep-alive',
        # 'Cookie': '_ym_uid=1670252833970654852; _ym_d=1670252833; _ym_isad=2; disclaimed=true',
        'Referer': 'https://bo.nalog.ru/organizations-card/10686195',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    params_get_org_id = {                   # Параметры запроса в общем виде
        'query': '6319038948',
        'page': '0',
    }
    params_get_org_id['query'] = str(inn)   # Параметры запроса, INN в данном случае
    response = requests.get(                # Выполнение запроса на сайт БФО - поиск по ИНН
        'https://bo.nalog.ru/nbo/organizations/search',
        params=params_get_org_id,
        cookies=cookies_get_org_id,
        headers=headers_get_org_id
    )

    # ID организации на сайте БФО
    org_id_json = response.json()    # Объект requests.Response модуля requests содержит всю информацию ответа сервера на HTTP-запрос requests.get()
    org_id = org_id_json['content'][0]['id']     # Объект requests.Response модуля requests содержит всю информацию ответа сервера на HTTP-запрос requests.get()
#    org_shortName = org_id_json['content'][0]['shortName']     # Объект requests.Response модуля requests содержит всю информацию ответа сервера на HTTP-запрос requests.get()
#    org_OGRN = org_id_json['content'][0]['ogrn']     # Объект requests.Response модуля requests содержит всю информацию ответа сервера на HTTP-запрос requests.get()
#    org_Reg = org_id_json['content'][0]['region']     # Объект requests.Response модуля requests содержит всю информацию ответа сервера на HTTP-запрос requests.get()
#    org_City = org_id_json['content'][0]['city']     # Объект requests.Response модуля requests содержит всю информацию ответа сервера на HTTP-запрос requests.get()
#    print('ИНН =', inn, '; org_id =', org_id, '; org_shortName =', org_shortName, '; org_OGRN =', org_OGRN, '; org_Reg =', org_Reg, '; org_City =', org_City)

#    print("\n", org_id_json, "\n")
    cookies_money_id = {
        '_ym_uid': '1670252833970654852',
        '_ym_d': '1670252833',
        '_ym_isad': '2',
        'disclaimed': 'true',
    }
    headers_money_id = {
        'Accept': '*/*',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-GB;q=0.8,en;q=0.7,en-US;q=0.6',
        'Connection': 'keep-alive',
        # 'Cookie': '_ym_uid=1670252833970654852; _ym_d=1670252833; _ym_isad=2; disclaimed=true',
        'Referer': 'https://bo.nalog.ru/organizations-card/10686195',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    
    year_money_id_resp = requests.get(  # Запрос на сайт БФО org_id - каких-то (средних) данных по организации в целом. Каких?
        f'https://bo.nalog.ru/nbo/organizations/{str(org_id)}/bfo/',
        cookies=cookies_money_id,
        headers=headers_money_id
    ).json()
    
    
    year_2019_money_id = None
    year_2020_money_id = None
    year_2021_money_id = None
    
    #для определения org_id в случае частичного отсутствия инфы    
    year_ALL_money_id = None

    for year_money_id in year_money_id_resp:
        if year_money_id['period'] == '2021':
            year_2021_money_id = year_money_id['id']
        if year_money_id['period'] == '2020':
            year_2020_money_id = year_money_id['id']
        if year_money_id['period'] == '2019':
            year_2019_money_id = year_money_id['id']

#    print('year_2021_money_id = ', year_2021_money_id)
#    print('year_2020_money_id = ', year_2020_money_id)
#    print('year_2019_money_id = ', year_2019_money_id)
    print('year_2021_money_id = ', year_2021_money_id, '; year_2020_money_id = ', year_2020_money_id, '; year_2019_money_id = ', year_2019_money_id)

    #нумерация ситуаций отсутствия инфы

    # Ситуации данных - на потом подумать: лучше битами на каждый год нумеровать.
    # 0 - есть все 3 года
    # 1 - нет 2021 г.
    # 2 - нет 2020 г.
    # 3 - нет 2019 г.               # на потом - 4
    # 4 - нет 2021 и 2020 гг.       # на потом - 3
    # 5 - нет 2021 и 2019 гг.
    # 6 - нет 2020 и 2019 гг.
    # 7 - нет 2021 и 2020 и 2019 гг.
    # На потом- заменить Error на SitCase. Это же не ошибка!
    year_ALL_money_id = year_2021_money_id
    if (year_2021_money_id == None):
        Error = 1
        year_ALL_money_id = year_2020_money_id

    if (year_2020_money_id == None):
        if (Error == 1):
            Error = 4
            year_ALL_money_id = year_2019_money_id
        else:
            Error = 2
            year_ALL_money_id = year_2019_money_id

    if (year_2019_money_id == None):
        if (Error == 1):
            Error = 5
            year_ALL_money_id = year_2020_money_id

        elif (Error == 2):
            Error = 6
            year_ALL_money_id = year_2021_money_id

        elif (Error == 4):
            Error = 7

        else:
            Error = 3
            year_ALL_money_id = year_2020_money_id

    print("Ситуация данных номер:", Error)
    cookies_year_money = {
        '_ym_uid': '1670252833970654852',
        '_ym_d': '1670252833',
        '_ym_isad': '2',
        'disclaimed': 'true',
    }
    headers_year_money = {
        'Accept': '*/*',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-GB;q=0.8,en;q=0.7,en-US;q=0.6',
        'Connection': 'keep-alive',
        # 'Cookie': '_ym_uid=1670252833970654852; _ym_d=1670252833; _ym_isad=2; disclaimed=true',
        'Referer': 'https://bo.nalog.ru/organizations-card/10686195',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }

    if (Error == 0 or Error == 2 or Error == 3 or Error == 6):
        response_year_money21 = requests.get(     # Запрос на сайт БФО по организации в 2021 году
            f'https://bo.nalog.ru/nbo/bfo/{str(year_2021_money_id)}/details',
            cookies=cookies_year_money,
            headers=headers_year_money
        ).json()       
    else:
        response_year_money21 = None

    
    if (Error == 0 or Error == 1 or Error == 3 or Error == 5): 
        response_year_money20 = requests.get(     # Запрос на сайт БФО по организации и 2020 году
            f'https://bo.nalog.ru/nbo/bfo/{str(year_2020_money_id)}/details',
            cookies=cookies_year_money,
            headers=headers_year_money
        ).json() 
    else:
        response_year_money20 = None 

    if (Error == 0 or Error == 1 or Error == 2 or Error == 4):
        response_year_money19 = requests.get(     # Запрос на сайт БФО по организации и 2019 году
            f'https://bo.nalog.ru/nbo/bfo/{str(year_2019_money_id)}/details',
            cookies=cookies_year_money,
            headers=headers_year_money
        ).json() 
    else:
        response_year_money19 = None
            
    cookies_okved = {
        '_ym_uid': '1670252833970654852',
        '_ym_d': '1670252833',
        '_ym_isad': '2',
        'disclaimed': 'true',
    }
    headers_okved = {
        'Accept': '*/*',
        'Accept-Language': 'ru-RU,ru;q=0.9,en-GB;q=0.8,en;q=0.7,en-US;q=0.6',
        'Connection': 'keep-alive',
        # 'Cookie': '_ym_uid=1670252833970654852; _ym_d=1670252833; _ym_isad=2; disclaimed=true',
        'Referer': 'https://bo.nalog.ru/organizations-card/10686195',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
    }
    if(Error != 7):
        okved_id = requests.get(     # Запрос на сайт БФО по организации - ОКВЭД-2
            f'https://bo.nalog.ru/nbo/bfo/{str(year_ALL_money_id)}/organizations/',
            cookies=cookies_okved,
            headers=headers_okved
        ).json()                            #['okved2_id']
        #okved_req = okved_pattern.replace('YEAR_MONEY_ID_ARG', str(year_2021_money_id))
        #okved_id = json.loads(os.popen(okved_req).read())['okved2_id']
        #print('ОКВЭД-2 =', okved_id)
        
    return response_year_money19, response_year_money20, response_year_money21, okved_id, org_id_json, year_money_id_resp # Добавка response


def BalFinResToDB(Balance, financialResult, capitalChange, power, inn, year):

    Strinn =[]
    Strinn.append(inn)
    Stryear = str(year)
    Strpower = []
    Strpower.append(power) #добавляется в CapitalChange
    
    querystring = f"SELECT inn FROM BalFinRes{year} WHERE inn = '{inn}'"
    cursor.execute(querystring)
    if (cursor.fetchval() == None):
        querystring = f"""INSERT INTO BalFinRes{year} 
                                (
                                    inn,
                                    1600_Active, 
                                    1100_Vneoborot, 
                                    1110_NematAct, 
                                    1120_ResResearch, 
                                    1130_NematResearch, 
                                    1140_MatResearch, 
                                    1150_OsnSredstva, 
                                    1160_VlojenieMat, 
                                    1170_FinVlojen, 
                                    1180_OtlNalog, 
                                    1190_ProchieNeoborot, 
                                    1200_Oborot, 
                                    1210_Zapas, 
                                    1220_NalogNDS, 
                                    1230_DebitDolg, 
                                    1240_FinVlogen, 
                                    1250_DenSredstva, 
                                    1260_ProchieOborot, 
                                    1700_Passive, 
                                    1300_CapitalRes, 
                                    1310_UstCapital, 
                                    1320_SelfAct, 
                                    1340_RecostVneoborot, 
                                    1350_AddCapital, 
                                    1360_ReservCapital, 
                                    1370_NeraspIncome, 
                                    1400_DolgObyaz, 
                                    1410_ZaemSredstva, 
                                    1420_OtlojNalogObyaz, 
                                    1430_OcenObyaz, 
                                    1450_ProchieObyaz, 
                                    1500_KratkObyaz, 
                                    1510_ZaeSredstva, 
                                    1520_CreditDolg, 
                                    1530_DohodBudPeriod, 
                                    1540_OcenObyaz, 
                                    1550_ProchieObyaz,
                                    2110_Viruchka,
                                    2120_Sebestoim,
                                    2100_ValPrib,
                                    2210_KomRashod,
                                    2220_UprRashod,
                                    2200_PribProd,
                                    2310_DohodUchast,
                                    2320_ProcPoluch,
                                    2330_ProcUplata,
                                    2340_ProchDohod,
                                    2350_ProchRashod,
                                    2300_PribDoNal,
                                    2410_NalPrib,
                                    2411_TekNalPrib,
                                    2421_PostNalObyaz,
                                    2412_OtlojNalPrib,
                                    2430_IzmNalObyaz,
                                    2450_IzmNalActiv,
                                    2460_Prochee,
                                    2400_ChistPrib,
                                    2510_ResPrcVneoborot,
                                    2520_ResProchOper,
                                    2530_NalPribNeChist,
                                    2500_SovFinRes,
                                    2900_BazPribAkc,
                                    2910_RazvPribAkc,
                                    3327_Dividends,
                                    Power
                               ) 
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,
                                 ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) """
        
        cursor.execute(querystring, Strinn + Balance + financialResult + capitalChange + Strpower)
        print(f"Данные добавлены в BalFinRes{year}.")

    else:
        print(f"В БД уже присутствуют данные за {year} год.")
    pass


def Parce_Data(response_year_money):

    BFO_Call_Balance_current = ['current1600', 'current1100', 'current1110', 'current1120', 'current1130', 'current1140', 
                            'current1150', 'current1160', 'current1170', 'current1180', 'current1190', 'current1200', 
                            'current1210', 'current1220', 'current1230', 'current1240', 'current1250', 'current1260', 
                            'current1700', 'current1300', 'current1310', 'current1320', 'current1340', 'current1350',
                            'current1360', 'current1370', 'current1400', 'current1410', 'current1420', 'current1430',
                            'current1450', 'current1500', 'current1510', 'current1520', 'current1530', 'current1540',
                            'current1550']

    BFO_Call_financialResult_current = ['current2110', 'current2120', 'current2100', 'current2210', 'current2220', 'current2200', 
                            'current2310', 'current2320', 'current2330', 'current2340', 'current2350', 'current2300', 
                            'current2410', 'current2411', 'current2421', 'current2412', 'current2430', 'current2450', 
                            'current2460', 'current2400', 'current2510', 'current2520', 'current2530', 'current2500',
                            'current2900', 'current2910']

    BFO_Call_capitalChange_current = ['total3327']
    
    Balance = [] * 37
    financialResult = [] * 26
    capitalChange = []
    
    Active_1600 = 0
    Viruchka_2110 = 0
    Pr_Dohod_2340 = 0
    Doh_dr_org_2310 = 0
    Proc_poluc_2320 = 0 
    
    for i in range (0, 37):
        
        try:
            BFO_Balance = None
            BFO_Balance = response_year_money[0]['balance'][BFO_Call_Balance_current[i]]
            if (BFO_Balance == 'null'):
                BFO_Balance = None;
        except KeyError:
            BFO_Balance = None;
        Balance.append(BFO_Balance) 
#        print(BFO_Call_Balance_current[i], ' = ', BFO_Balance, '\n')
        
        if (i == 0):
            if (BFO_Balance != None):
                Active_1600 = int(BFO_Balance)
            else:
                Active_1600 = 0
        
        BFO_Balance = None
    
    for j in range (0, 26):
        
        try:
            BFO_financialResult = None
            BFO_financialResult = response_year_money[0]['financialResult'][BFO_Call_financialResult_current[j]]
            if (BFO_financialResult == 'null'):
                BFO_financialResult = None;
        except KeyError:
            BFO_inancialResult = None;
        financialResult.append(BFO_financialResult)
        
#Определение мощи
        if (j == 0):
            if (BFO_financialResult != None):
                Viruchka_2110 = int(BFO_financialResult)
            else:
                Viruchka_2110 = 0
        if (j == 9):
            if (BFO_financialResult != None):
                Pr_Dohod_2340 = int(BFO_financialResult)
            else:
                Pr_Dohod_2340 = 0
        if (j == 6):
            if (BFO_financialResult != None):
                Doh_dr_org_2310 = int(BFO_financialResult)
            else:
                Doh_dr_org_2310 = 0
        if (j == 7):
            if (BFO_financialResult != None):
                Proc_poluc_2320 = int(BFO_financialResult)
            else:
                Proc_poluc_2320 = 0
        
        BFO_financialResult = None
    
    try:
        BFO_capitalChange = None
        BFO_capitalChange = response_year_money[0]['capitalChange'][BFO_Call_capitalChange_current[0]]
        if (BFO_capitalChange == 'null'):
            BFO_capitalChange = None;
    except KeyError:
        BFO_capitalChange = None;
    capitalChange.append(BFO_capitalChange)
#    print(BFO_Call_capitalChange_current[0], ' = ', BFO_capitalChange, '\n')
    BFO_capitalChange = None
    
#Вычисление мощи
    power = str(max(Active_1600, Viruchka_2110 + Pr_Dohod_2340 + Doh_dr_org_2310 + Proc_poluc_2320))
#    capitalChange.append(power) #добавляется в CapitalChange

    return Balance, financialResult, capitalChange, power
        

def Parce_Org(okved_id, Data_Org, inn):
    
    Data = []
    Adr = []
    Nm = []

    okved2_id = okved_id['okved2_id'] 
    Full_Name = okved_id['fullName']
    address = okved_id['address']
#    Adr = address.split(", ")
#    if (len(Adr[0]) > 30):
#        Adr = address.split(",")
    
    Nm = Full_Name.split('\"')
    if (Nm[0] == Full_Name):
        Short_Name = Full_Name
    else:
        Short_Name = Nm[1]
        
#    Region = Adr[1]
#    City = Adr[2]

    

    Data.append(inn)
    Data.append(Full_Name)
    Data.append(Short_Name)
    Data.append(address)
#    Data.append(Region)
    Data.append(None)
#    Data.append(City)
    Data.append(None)
    Data.append(okved2_id)
    
    print("fullName:", Full_Name)
    print("N:", Short_Name)
    print("okved2_id:", okved2_id)
    print("address:", address)
#    print("R:", Region)
#    print("C:", City)
#    print("\n")
        
    querystring = f"SELECT inn FROM org WHERE inn = '{inn}'"
    cursor.execute(querystring)
    if (cursor.fetchval() == None):
        querystring = f"""INSERT INTO org 
                                (
                                inn,
                                OrgName,
                                Short_Name,
                                OrgAddress,
                                Region,
                                City,
                                okved2,
                                Мощь,
                                2110_Viruchka_2019,
                                2110_Viruchka_2020,
                                2110_Viruchka_2021,
                                2400_ChistPrib_2019,
                                2400_ChistPrib_2020,
                                2400_ChistPrib_2021,
                                1600_Active_2019,
                                1600_Active_2020,
                                1600_Active_2021,
                                Neadekv,
                                comment
                               ) 
                                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?) """
        
        cursor.execute(querystring, Data + Data_Org + [False] + [None])
        print(f"Данные добавлены в org.")

    else:
        print(f"В БД уже присутствуют данные.")
    pass
        

def DownloadData(Name_Org_Actual, Name_Org_Bad, Name_Org_Bad_inn, Data_org):

    amount = len(open(Name_Org_Actual).readlines())
    print("Количество компаний:", amount,'\n')

    count = 0

    f = open(Name_Org_Actual)
    o = open(Name_Org_Bad, 'a')
    a = open(Name_Org_Bad_inn, 'a')
    data = open(Data_org, 'a')

    for line in f:

        SFinOne = time.time()

        Data_Org = []
        line = line.rstrip()
        count = count + 1
        
        Balance_21 = None
        Balance_20 = None
        Balance_19 = None
        financialResult_21 = None
        financialResult_20 = None
        financialResult_19 = None
        capitalChange_21 = None
        capitalChange_20 = None
        capitalChange_19 = None
        power_21 = 0
        power_20 = 0
        power_19 = 0
        Viruchka_2110_21 = None
        Active_1600_21 = None
        ChistPrib_2400_21 = None
        Viruchka_2110_20 = None
        Active_1600_20 = None
        ChistPrib_2400_20 = None
        Viruchka_2110_19 = None
        Active_1600_19 = None
        ChistPrib_2400_19 = None
        okved_id = None
        
        if len(line) < 10:
            line = '0' * (10 - len(line)) + line
         
        try:
            #Извлечение .json данных из метода доставания
            response_year_money19, response_year_money20, response_year_money21, okved_id, org_id_json, year_money_id_resp = get_external_data_by_inn(line)
            
            #распарсивание данных и загрузка их в БД
            if(response_year_money21 != None):
                Balance_21, financialResult_21, capitalChange_21, power_21 = Parce_Data(response_year_money21)
                BalFinResToDB(Balance_21, financialResult_21, capitalChange_21, power_21, line, 2021)
                
                Viruchka_2110_21 = financialResult_21[0]
                Active_1600_21 = Balance_21[0]
                ChistPrib_2400_21 = financialResult_21[19]
                
            if(response_year_money20 != None):
                Balance_20, financialResult_20, capitalChange_20, power_20 = Parce_Data(response_year_money20)
                BalFinResToDB(Balance_20, financialResult_20, capitalChange_20, power_20, line, 2020)
                
                Viruchka_2110_20 = financialResult_20[0]
                Active_1600_20 = Balance_20[0]
                ChistPrib_2400_20 = financialResult_20[19]
                
            if(response_year_money19 != None):
                Balance_19, financialResult_19, capitalChange_19, power_19 = Parce_Data(response_year_money19)
                BalFinResToDB(Balance_19, financialResult_19, capitalChange_19, power_19, line, 2019)
                
                Viruchka_2110_19 = financialResult_19[0]
                Active_1600_19 = Balance_19[0]
                ChistPrib_2400_19 = financialResult_19[19]
            
            Power_All = str(max(int(power_21), int(power_20), int(power_19)))
            
      #      Data_Org =[Active_1600_19, Active_1600_20, Active_1600_21, Viruchka_2110_19, Viruchka_2110_20, Viruchka_2110_21,
      #                 ChistPrib_2400_19, ChistPrib_2400_20, ChistPrib_2400_21, Power_All]
            Data_Org =[Power_All, Viruchka_2110_19, Viruchka_2110_20, Viruchka_2110_21, 
                       ChistPrib_2400_19, ChistPrib_2400_20, ChistPrib_2400_21, Active_1600_19, 
                       Active_1600_20, Active_1600_21]
            
            #запись данных в бд
            Parce_Org(okved_id, Data_Org, line)
            
            #коммит SQL        
            connection_sql.commit()
            
            #таймер
            SFin = time.time()                  # Тек. время
            SFinOneFin = SFin - SFinOne         # Время работы на тек. организацию
            S = SFin - SInit                    # Время работы от начала запуска
            S_Avg = S/count                     # Среднее время на одну организацию
            S_Rest = round( (amount - count) * S_Avg / 60, 0) # Время до конца работы, мин
            print(f"#{count}/{amount} ИНН: {line} - Complete")
            print("Время работы:", round(S/60, 0), " мин.; Осталось:", S_Rest, " мин.; Среднее время на организацию:", round(S_Avg, 3), " сек.; на эту организацию:", round(SFinOneFin, 3), " сек.")
            # print("\n")
            print("------------------------------")


            data.write(f"{org_id_json} \n {okved_id} \n {year_money_id_resp} \n {response_year_money19} \n {response_year_money20} \n {response_year_money21} \n \n \n")
            
        except Exception as err:
            print(err)
            if(okved_id == None):
                print(f"#{count}/{amount} ИНН: {line} - Error")
                print("Организация не найдена\n")
                o.write(f"#{count}/{amount} ИНН: {line} - организация не найдена \n")
            else:
                print(f"#{count}/{amount} ИНН: {line} - Error")
                o.write(f"#{count}/{amount} ИНН: {line} - Error {err}, \n")
                a.write(f"{line} \n")
        
    f.close()
    o.close()
    a.close()
    data.close()
        
    pass

DownloadData(Name_Org_Actual, Name_Org_Bad, Name_Org_Bad_inn, Data_org)


ModuleNotFoundError: No module named 'pyodbc'